In [6]:
pip install tenacity

  Obtaining dependency information for tenacity from https://files.pythonhosted.org/packages/f4/f1/990741d5bb2487d529d20a433210ffa136a367751e454214013b441c4575/tenacity-8.2.3-py3-none-any.whl.metadata

[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [62]:
import requests
from bs4 import BeautifulSoup
from tenacity import retry, stop_after_attempt,wait_exponential
import urllib
import time
import numpy as np
import pandas as pd

#tenacityのライブラリに関する記事は下記
#https://zenn.dev/taroman_zenn/articles/dd0b33a3a37d1e

In [63]:
#Step1必要な情報を取得する

#SUUMO上で必要な情報を検索する
#今回はペルソナに合致した下記の条件を設定｜ペルソナ：https://www.dropbox.com/scl/fi/pdo0x2sm6es6n76sek2i7/Tech0_STEP3-1_-_v1.0.pdf?rlkey=2attz3z14yyoanwm53spxk6t1&dl=0
#検索条件：東京都内、20万円以下（管理費・共益費込み）、2K以上の間取り、駅徒歩10分以内、バストイレ別、室内洗濯機置場
#取得したURL：https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=50&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&ta=13&cb=0.0&ct=20.0&co=1&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&et=10&mb=0&mt=9999999&cn=9999999&tc=0400301&tc=0400501&fw2=

#複数ページの情報をまとめて取得するリストを作成
data_samples = []

#スクレイピングするページ数を指定
#1ページの表示件数を最大の50件に設定した上で、出力されたページ数237を上回るように設定する
max_page = 240

#SUUMOで自分が調べたい地域を指定して検索し、出力したページのURL
URL = 'https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&pc=50&smk=&po1=25&po2=99&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sc=13101&sc=13102&sc=13103&sc=13104&sc=13105&sc=13113&sc=13106&sc=13107&sc=13108&sc=13118&sc=13121&sc=13122&sc=13123&sc=13109&sc=13110&sc=13111&sc=13112&sc=13114&sc=13115&sc=13120&sc=13116&sc=13117&sc=13119&ta=13&cb=0.0&ct=20.0&co=1&md=05&md=06&md=07&md=08&md=09&md=10&md=11&md=12&md=13&md=14&et=10&mb=0&mt=9999999&cn=9999999&tc=0400301&tc=0400501&fw2=&page={:d}'
suumo_URL ='https://suumo.jp/'

#試行回数3回、リトライ間の初期待機時間10秒、連続するリトライの時間を100秒にする（指数関数的に増やすが、100秒までで制限する）
#Pythonでは、デコレータの引数を|で結合することはできないため、,で指定すること
@retry(stop=stop_after_attempt(3),wait=wait_exponential(multiplier=10, min=10, max=100))
def load_page(URL):
    html = requests.get(URL)
    soup = BeautifulSoup(html.content, 'html.parser')
    return soup


In [64]:
# 処理時間を測りたい
start = time.time()
times = []

#関数については下記を参考にすること
#enumerate()関数 https://note.nkmk.me/python-enumerate-start/
#strip()関数 https://note.nkmk.me/python-str-remove-strip/#strip

for page in range(1,max_page+1):
    before = time.time()
    #ページ情報
    soup = load_page(URL.format(page))
    #物件リストを指定
    mother = soup.find_all(class_='cassetteitem')
    
    #物件ごとの処理
    for child in mother:
        
        #建物情報
        data_home = []
        #カテゴリ
        data_home.append(child.find(class_='ui-pct ui-pct--util1').text)
        #建物名
        data_home.append(child.find(class_='cassetteitem_content-title').text)
        #住所
        data_home.append(child.find(class_='cassetteitem_detail-col1').text)
        #最寄駅のアクセス
        children = child.find(class_='cassetteitem_detail-col2')
        for id, grandchild in enumerate(children.find_all(class_='cassetteitem_detail-text')):
            data_home.append(grandchild.text)
        #築年数と階数
        children = child.find(class_='cassetteitem_detail-col3')
        for grandchild in children.find_all('div'):
            data_home.append(grandchild.text)
        
        #部屋情報
        rooms = child.find(class_='cassetteitem_other')
        for room in rooms.find_all(class_='js-cassette_link'):
            data_room = []
            
            #部屋情報が入っている表を探索
            for id_, grandchild in enumerate(room.find_all('td')):
                #階
                if id_ == 2:
                    data_room.append(grandchild.text.strip())
                #家賃と管理費
                elif id_ == 3:
                    data_room.append(grandchild.find(class_='cassetteitem_other-emphasis ui-text--bold').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--administration').text)
                #敷金と礼金
                elif id_ == 4:
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--deposit').text)
                    data_room.append(grandchild.find(class_='cassetteitem_price cassetteitem_price--gratuity').text)
                #間取りと面積
                elif id_ == 5:
                    data_room.append(grandchild.find(class_='cassetteitem_madori').text)
                    data_room.append(grandchild.find(class_='cassetteitem_menseki').text)
                #url
                elif id_ == 8:
                    get_url = grandchild.find(class_='js-cassette_link_href cassetteitem_other-linktext').get('href')
                    abs_url = urllib.parse.urljoin(suumo_URL, get_url)
                    data_room.append(abs_url)
            #物件情報と部屋情報を結合する
            data_sample = data_home + data_room,
            data_samples.append(data_sample)

    #1アクセスごとに1秒休む
    time.sleep(1)
    
    #進捗確認
    #このページの作業時間を表示
    after = time.time()
    running_time = after - before
    times.append(running_time)
    print(f'総取得件数:{len(data_samples)}')
    #作業進捗
    complete_ratio = round(page/max_page*100,3)
    print(f'完了:{complete_ratio}%')
    #作業の残り時間目安を表示
    running_mean = np.mean(times)
    running_required_time = running_mean * (max_page - page)
    hour = int(running_required_time/3600)
    minute = int((running_required_time%3600)/60)
    second = int(running_required_time%60)
    print(f'残り時間:{hour}時間{minute}分{second}秒\n')

    
# 処理時間を測りたい
finish = time.time()
running_all = finish - start
print('総経過時間：',running_all)
sound()

総取得件数:176
完了:0.417%
残り時間:0時間20分53秒

総取得件数:352
完了:0.833%
残り時間:0時間25分20秒

総取得件数:528
完了:1.25%
残り時間:0時間21分45秒

総取得件数:704
完了:1.667%
残り時間:0時間20分31秒

総取得件数:880
完了:2.083%
残り時間:0時間18分58秒

総取得件数:1056
完了:2.5%
残り時間:0時間18分10秒

総取得件数:1232
完了:2.917%
残り時間:0時間17分32秒

総取得件数:1408
完了:3.333%
残り時間:0時間17分3秒

総取得件数:1584
完了:3.75%
残り時間:0時間16分46秒

総取得件数:1760
完了:4.167%
残り時間:0時間18分33秒

総取得件数:1936
完了:4.583%
残り時間:0時間18分5秒

総取得件数:2112
完了:5.0%
残り時間:0時間17分37秒

総取得件数:2288
完了:5.417%
残り時間:0時間17分13秒

総取得件数:2464
完了:5.833%
残り時間:0時間16分51秒

総取得件数:2640
完了:6.25%
残り時間:0時間16分32秒

総取得件数:2816
完了:6.667%
残り時間:0時間16分18秒

総取得件数:2992
完了:7.083%
残り時間:0時間16分2秒

総取得件数:3168
完了:7.5%
残り時間:0時間15分47秒

総取得件数:3344
完了:7.917%
残り時間:0時間20分51秒

総取得件数:3520
完了:8.333%
残り時間:0時間20分22秒

総取得件数:3696
完了:8.75%
残り時間:0時間19分56秒

総取得件数:3872
完了:9.167%
残り時間:0時間19分36秒

総取得件数:4048
完了:9.583%
残り時間:0時間19分11秒

総取得件数:4224
完了:10.0%
残り時間:0時間18分51秒

総取得件数:4400
完了:10.417%
残り時間:0時間18分32秒

総取得件数:4576
完了:10.833%
残り時間:0時間18分14秒

総取得件数:4752
完了:11.25%
残り時間:0時間17分58秒

総取得件数:4928
完了:11.6

In [66]:
data_samples

[(['賃貸一戸建て',
   '奥山方',
   '東京都荒川区西日暮里２',
   'ＪＲ山手線/日暮里駅 歩5分',
   'ＪＲ山手線/西日暮里駅 歩10分',
   'ＪＲ常磐線/三河島駅 歩9分',
   '築41年',
   '2階建',
   '1階',
   '15万円',
   '-',
   '15万円',
   '15万円',
   '4K',
   '73.75m2',
   'https://suumo.jp/chintai/jnc_000086740977/?bc=100355480324'],),
 (['賃貸マンション',
   '都営大江戸線 東中野駅 3階建 築21年',
   '東京都新宿区北新宿３',
   '都営大江戸線/東中野駅 歩9分',
   '東京メトロ東西線/落合駅 歩15分',
   '東京メトロ丸ノ内線/中野坂上駅 歩14分',
   '築21年',
   '3階建',
   '1階',
   '19万円',
   '10000円',
   '-',
   '19万円',
   '4SK',
   '80m2',
   'https://suumo.jp/chintai/jnc_000086029265/?bc=100355326785'],),
 (['賃貸マンション',
   'kinomori代々木上原',
   '東京都渋谷区上原２',
   '東京メトロ千代田線/代々木公園駅 歩9分',
   '小田急線/代々木八幡駅 歩8分',
   '小田急線/代々木上原駅 歩7分',
   '築1年',
   '4階建',
   '4階',
   '12.7万円',
   '10000円',
   '-',
   '-',
   '2K',
   '23.7m2',
   'https://suumo.jp/chintai/jnc_000086724239/?bc=100355532143'],),
 (['賃貸マンション',
   'kinomori代々木上原',
   '東京都渋谷区上原２',
   '東京メトロ千代田線/代々木公園駅 歩9分',
   '小田急線/代々木八幡駅 歩8分',
   '小田急線/代々木上原駅 歩7分',
   '築1年',
   '4階建',
   '1階',
   '13万

In [67]:
#Step2データを整形する

#リスト内包表記を使用し、data_samplesリストの各要素からアイテムを取得し、新しいリスト split_data_samplesへ格納する
#item[0]となっているのは、タプル内の要素が1つのリストのみであるため＝([],)となっている
split_data_samples = [item[0] for item in data_samples]

# DataFrame の作成
df = pd.DataFrame(split_data_samples)

df.columns = ['カテゴリ', '物件名', '所在地', 'アクセス1', 'アクセス2', 'アクセス3', '築年数', '建物の階数', '部屋の階数', '賃料', '管理費', '敷金', '礼金', '間取り', '面積', 'URL']
df

,カテゴリ,物件名,所在地,アクセス1,アクセス2,アクセス3,築年数,建物の階数,部屋の階数,賃料,管理費,敷金,礼金,間取り,面積,URL
0,賃貸一戸建て,奥山方,東京都荒川区西日暮里２,ＪＲ山手線/日暮里駅 歩5分,ＪＲ山手線/西日暮里駅 歩10分,ＪＲ常磐線/三河島駅 歩9分,築41年,2階建,1階,15万円,-,15万円,15万円,4K,73.75m2,https://suumo.jp/chintai/jnc_000086740977/?bc=...
1,賃貸マンション,都営大江戸線 東中野駅 3階建 築21年,東京都新宿区北新宿３,都営大江戸線/東中野駅 歩9分,東京メトロ東西線/落合駅 歩15分,東京メトロ丸ノ内線/中野坂上駅 歩14分,築21年,3階建,1階,19万円,10000円,-,19万円,4SK,80m2,https://suumo.jp/chintai/jnc_000086029265/?bc=...
2,賃貸マンション,kinomori代々木上原,東京都渋谷区上原２,東京メトロ千代田線/代々木公園駅 歩9分,小田急線/代々木八幡駅 歩8分,小田急線/代々木上原駅 歩7分,築1年,4階建,4階,12.7万円,10000円,-,-,2K,23.7m2,https://suumo.jp/chintai/jnc_000086724239/?bc=...
3,賃貸マンション,kinomori代々木上原,東京都渋谷区上原２,東京メトロ千代田線/代々木公園駅 歩9分,小田急線/代々木八幡駅 歩8分,小田急線/代々木上原駅 歩7分,築1年,4階建,1階,13万円,10000円,-,-,2K,24.08m2,https://suumo.jp/chintai/jnc_000086724226/?bc=...
4,賃貸マンション,kinomori代々木上原,東京都渋谷区上原２,東京メトロ千代田線/代々木公園駅 歩9分,小田急線/代々木八幡駅 歩8分,小田急線/代々木上原駅 歩7分,築1年,4階建,2階,13.1万円,10000円,-,-,2K,24.08m2,https://suumo.jp/chintai/jnc_000086724228/?bc=...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41943,賃貸マンション,レジディア浅草橋,東京都台東区蔵前１,都営浅草線/蔵前駅 歩3分,ＪＲ総武線/浅草橋駅 歩5分,都営大江戸線/新御徒町駅 歩16分,築19年,12階建,5階,14.5万円,15000円,14.5万円,14.5万円,2K,40.54m2,https://suumo.jp/chintai/jnc_000086398901/?bc=...
41944,賃貸マンション,レジディア浅草橋,東京都台東区蔵前１,都営浅草線/蔵前駅 歩3分,ＪＲ総武線/浅草橋駅 歩5分,都営大江戸線/新御徒町駅 歩16分,築19年,12階建,5階,14.9万円,15000円,14.9万円,14.9万円,2K,40.54m2,https://suumo.jp/chintai/jnc_000085340930/?bc=...
41945,賃貸マンション,レジディア浅草橋,東京都台東区蔵前１,都営浅草線/蔵前駅 歩3分,ＪＲ総武線/浅草橋駅 歩5分,都営大江戸線/新御徒町駅 歩16分,築19年,12階建,11階,15.4万円,15000円,15.4万円,15.4万円,2K,40.54m2,https://suumo.jp/chintai/jnc_000085377038/?bc=...
41946,賃貸マンション,レジディア浅草橋,東京都台東区蔵前１,都営浅草線/蔵前駅 歩3分,ＪＲ総武線/浅草橋駅 歩5分,都営大江戸線/新御徒町駅 歩16分,築19年,12階建,3階,15.5万円,15000円,15.5万円,-,2K,40.54m2,https://suumo.jp/chintai/jnc_000086252981/?bc=...


In [68]:
#Step3
#出力した結果をCSVファイルに保存する
suumo_df = df
suumo_df.to_csv('suumo_data.csv', index=False, encoding='utf-8-sig')